In [ ]:
# default_exp data_collection_linkedin

# data_collection_linkedin

> API details.

In [ ]:
#hide
# from nbdev.showdoc import *

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import re, time, requests
import pandas as pd
from parsel import Selector
import config

In [ ]:
def strip_html(data):
    p = re.compile(r'<.*?>')
    if isinstance(data, str):
        return p.sub('', data)
    else: return ""

def get_criteria(criteria):    
    result = {Selector(item).css("h3::text").get():Selector(item).css("span::text").getall() for item in criteria}
    return result

def get_job_detail(job_url, published):
    """
    This function parse a job url and return job detailed information
    """
    selector = Selector(requests.get(job_url).text)
    # Get job title, company url
    top_card_css = 'div[class*="topcard__content-left"]'
    title = selector.css(top_card_css).xpath('./h1/text()').get()
    company_url = selector.css(top_card_css).xpath('./h3/span/a/@href').get()
    # Get job description
    desc = selector.css('div[class*="description__text"]').get()
    desc = strip_html(desc)
    # Get job criteria
    criteria = selector.css('ul[class*="job-criteria__list"]').xpath("li").getall()
    criteria = get_criteria(criteria)
    return (title, company_url, published, desc, criteria)    


In [ ]:
jobs_list = pd.read_csv(config.LINKEDIN_JOBLIST)
jobs_list.shape

(377, 4)

In [ ]:
jobs_list = jobs_list[jobs_list.collected == 0]

In [ ]:
jobs_list.head()

,title,published,url,collected
0,Data Scientist,2021-05-07,https://au.linkedin.com/jobs/view/data-scienti...,0
1,Data Scientist,2021-05-06,https://au.linkedin.com/jobs/view/data-scienti...,0
2,Data Scientist - Artificial Intelligence/Machi...,2021-05-07,https://au.linkedin.com/jobs/view/data-scienti...,0
3,Entry level Data Scientist / Risk Analyst oppo...,2021-05-07,https://au.linkedin.com/jobs/view/entry-level-...,0
4,Data Scientist,2021-05-06,https://au.linkedin.com/jobs/view/data-scienti...,0


In [ ]:
jobs_list.isna().sum()

title        0
published    0
url          0
collected    0
dtype: int64

### Get all jobs

In [ ]:
jobs = [get_job_detail(url, published) for url, published in zip(jobs_list.url, jobs_list.published)]
print(len(jobs))

377


In [ ]:
# Update collected status to 1
jobs_list.collected = 1

### Save data

In [ ]:
jobs = pd.DataFrame(jobs, columns=['title','company_url', 'published', 'description', 'criteria'])
jobs.to_csv(config.LINKEDIN_JOBS, mode="a", index=False)

In [ ]:
# TESTING
# url = "https://www.linkedin.com/jobs/view/lead-researcher-jira-work-management-at-atlassian-2469567032/?trackingId=DTQQrEPqTTdVIHYgcOaGgg%3D%3D&refId=CGhTPUgib1AcmehqhqVpoQ%3D%3D&pageNum=0&position=1&trk=public_jobs_jserp-result_search-card&originalSubdomain=au"
# get_job_detail(url)